In [19]:
import os
import json
import time
from glob import glob
from pathlib import Path
from termcolor import colored

from pytket import Circuit
from pytket.qasm import circuit_from_qasm_str
from pytket.extensions.cutensornet import TensorNetwork
import cuquantum as cq
import numpy as np

In [20]:
def load_circuit(file):
    """Load a circuit from a tket1 json or qasm file."""
    file = Path(file)
    # Check the extension
    if file.suffix == ".json":
        with open(file, "r") as f:
            json_circ = json.load(f)
            # Ignore classical registers
            json_circ["bits"] = []
            circ = Circuit.from_dict(json_circ)
    elif file.suffix == ".qasm":
        with open(file, "r") as f:
            # Ignore classical registers
            qasm = "".join(line for line in f if "creg" not in line)
            circ = circuit_from_qasm_str(qasm)
    else:
        raise ValueError(f"Unknown file extension: {file.suffix}")

    return circ

In [21]:
correct_file = "bef/3ad505ce-d668-11ea-a4be-38f9d36dfbf2.json"
invalid_file = "aft/3ad505ce-d668-11ea-a4be-38f9d36dfbf2-hypertket-best-final-circ.json"

correct = load_circuit(correct_file)
invalid = load_circuit(invalid_file)

correct_u = correct.get_unitary()
invalid_u = invalid.get_unitary()

In [34]:
print("Correct unitary:")
print(correct_u)

print()
print("Invalid unitary:")
print(invalid_u)

print()
print("Correct == Invalid?")
print(np.isclose(correct_u, invalid_u, atol=1e-14, rtol=0))

Correct unitary:
[[-0.32160533+0.4809864j   0.34192845-0.08740994j -0.08471252-0.33137674j
   0.54109469+0.36179596j]
 [ 0.06357949+0.34715012j -0.3024334 +0.49326661j  0.55490954+0.34022814j
   0.33643727-0.06161747j]
 [-0.04557231-0.0743281j  -0.13153023+0.71816769j -0.65555182-0.12006233j
   0.10500273-0.06437965j]
 [-0.70736533-0.18082952j -0.04846125-0.07247766j  0.10238861-0.06846082j
   0.16506329-0.64569131j]]

Invalid unitary:
[[-0.32160533+0.4809864j   0.34192845-0.08740994j -0.08471252-0.33137674j
   0.54109469+0.36179596j]
 [ 0.06357949+0.34715012j -0.3024334 +0.49326661j  0.55490954+0.34022814j
   0.33643727-0.06161747j]
 [-0.04557231-0.0743281j  -0.13153023+0.71816769j -0.65555182-0.12006233j
   0.10500273-0.06437965j]
 [-0.70736533-0.18082952j -0.04846125-0.07247766j  0.10238861-0.06846082j
   0.16506329-0.64569131j]]

Correct == Invalid?
[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [37]:
print("The values are almost equal:")
print("Correct:", correct_u[0][0])
print("Invalid:", invalid_u[0][0])

The values are almost equal:
Correct: (-0.3216053271232092+0.4809864038808974j)
Invalid: (-0.3216053271232111+0.48098640388089564j)
